# DCGAN

Deep Convolutional Generatice Adversarial Network

![](https://www.researchgate.net/publication/331282441/figure/fig3/AS:729118295478273@1550846756282/Deep-convolutional-generative-adversarial-networks-DCGAN-for-generative-model-of-BF-NSP.png)

## Preparation

### Load the Library

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

tf.__version__

'2.7.4'

### Prepare Training Data

In [32]:
(train_x, _), (_, _) = tf.keras.datasets.mnist.load_data()

# Preprocessing
train_x = train_x / 127.5 - 1
train_x = np.expand_dims(train_x, axis=3)

### Define a Global Variables

In [5]:
INPUT_SHAPE = train_x.shape
DATA_SHAPE = INPUT_SHAPE[1:]
DATA_ROWS = DATA_SHAPE[0]
DATA_COLS = DATA_SHAPE[1]
DATA_CHANNELS = DATA_SHAPE[2]
DATA_SIZE = np.prod(DATA_SHAPE)

LATENT_Z_DIMS = 100

EPOCHS = 10000
BATCH_SIZE = 128
NUM_DISPLAY_LOG = 20
EPOCH_DISPLAY_LOG = EPOCHS // NUM_DISPLAY_LOG

REAL_LABELS, FAKE_LABELS = np.ones((BATCH_SIZE, 1)), np.zeros((BATCH_SIZE, 1))

## Define Models

### Generative Model

In [10]:
def build_generator():
    model = tf.keras.models.Sequential([
        # Input: Latent Z data
        tf.keras.layers.Dense(units=7*7*128, input_dim=LATENT_Z_DIMS),
        tf.keras.layers.Reshape((7,7,128)),
        
        # DC Conv2D #1
        tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(3,3), strides=(2,2), 
                                        padding="same"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.01),
        
        # DC Conv2D #2
        tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=(3,3), padding="same"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(alpha=0.01),
        
        # DC Conv2D Last
        tf.keras.layers.Conv2DTranspose(filters=1, kernel_size=(3,3), strides=(2,2),
                                        padding="same", activation="tanh")
    ])
    return model

### Discriminative Model

In [7]:
def build_discriminator():
    model = tf.keras.models.Sequential([
        # Convolutional Layers
        tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), 
                               padding="same", input_shape=DATA_SHAPE),
        tf.keras.layers.LeakyReLU(alpha=0.01),
        
        tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(2,2), padding="same"),
        tf.keras.layers.LeakyReLU(alpha=0.01),
        
        tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(2,2), padding="same"),
        tf.keras.layers.LeakyReLU(alpha=0.01),
        
        tf.keras.layers.Flatten(),
        
        # Full-Connected Layers
        tf.keras.layers.Dense(units=256, activation="relu"),
        tf.keras.layers.Dense(units=1, activation="sigmoid")
    ])
    return model

### DCGAN Model

#### Define

In [12]:
def build_dcgan(generator, discriminator):
    model = tf.keras.models.Sequential([
        generator,
        discriminator
    ])
    return model

#### Implement

In [13]:
discriminator = build_discriminator()
discriminator.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
discriminator.trainable = False 

generator = build_generator()

dcgan = build_dcgan(generator, discriminator)
dcgan.compile(optimizer="adam", loss="binary_crossentropy")
dcgan.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 28, 28, 1)         726401    
                                                                 
 sequential_4 (Sequential)   (None, 1)                 1418753   
                                                                 
Total params: 2,145,154
Trainable params: 726,209
Non-trainable params: 1,418,945
_________________________________________________________________


## Training DCGAN Model

In [34]:
def get_discriminator_train_data():
    real_idx = np.random.randint(0, INPUT_SHAPE[0], BATCH_SIZE)
    real_images = train_x[real_idx]
    latent_data = np.random.normal(0, 1, (BATCH_SIZE, LATENT_Z_DIMS))
    fake_images = generator.predict(latent_data, verbose=0)
    
    batch_x = np.vstack([real_images, fake_images])
    batch_y = np.vstack([REAL_LABELS, FAKE_LABELS])
    shuffle_idx = np.arange(batch_x.shape[0])
    np.random.shuffle(shuffle_idx)
    batch_x = batch_x[shuffle_idx]
    batch_y = batch_y[shuffle_idx]
    return batch_x, batch_y

def get_generator_train_data():
    return np.random.normal(0, 1, (BATCH_SIZE, LATENT_Z_DIMS)), REAL_LABELS

def get_eta(num_curr, time_curr):
    return datetime.now() + (NUM_DISPLAY_LOG - num_curr) * time_curr

In [36]:
def train():
    start_train = datetime.now()
    print(f"Start Training - Data Shape: {INPUT_SHAPE}, "
            f"Epochs: {EPOCHS}, Batch Size: {BATCH_SIZE}, "
            f"Time: {start_train}")
    
    losses = []
    accuracies = []
    
    start_loop = datetime.now()
    for epoch in range(EPOCHS):
        x, y = get_discriminator_train_data()
        d_loss, d_accuracy = discriminator.train_on_batch(x, y)
        x, y = get_generator_train_data()
        g_loss = dcgan.train_on_batch(x, y)
        
        if (epoch+1) % EPOCH_DISPLAY_LOG == 0:
            losses.append((d_loss, g_loss))
            accuracies.append(d_accuracy*100)
            
            end_loop = datetime.now()
            num_curr_loop = (epoch+1) // EPOCH_DISPLAY_LOG
            print(f"{epoch+1: >6}({num_curr_loop: >2}/{NUM_DISPLAY_LOG}), "
                  f"[D loss: {d_loss:8.5f}, accuracy: {d_accuracy:8.5f}], "
                  f"[G loss: {g_loss:8.5f}], "
                  f"Time[(Curr){end_loop-start_loop}, (Total){end_loop-start_train}, "
                  f"(ETA){get_eta(num_curr_loop, end_loop-start_loop)}]")
            start_loop = end_loop
            
    end_train = datetime.now()
    print(f"End Training - Time: {end_train}, "
          f"Total Processing Time: {end_train-start_train}")

In [37]:
train()

Start Training - Data Shape: (60000, 28, 28, 1), Epochs: 10000, Batch Size: 128, Time: 2022-10-07 21:39:29.523741
   500( 1/20), [D loss:  0.23503, accuracy:  0.90625], [G loss:  2.88385], Time[(Curr)0:00:27.182249, (Total)0:00:27.182291, (ETA)2022-10-07 21:48:33.168784]
  1000( 2/20), [D loss:  0.26363, accuracy:  0.87891], [G loss:  3.16406], Time[(Curr)0:00:26.803939, (Total)0:00:53.986230, (ETA)2022-10-07 21:48:25.980893]
  1500( 3/20), [D loss:  0.17858, accuracy:  0.92969], [G loss:  2.72501], Time[(Curr)0:00:27.294675, (Total)0:01:21.280905, (ETA)2022-10-07 21:48:34.814143]
  2000( 4/20), [D loss:  0.18563, accuracy:  0.91797], [G loss:  3.75814], Time[(Curr)0:00:28.574728, (Total)0:01:49.855633, (ETA)2022-10-07 21:48:56.575044]
  2500( 5/20), [D loss:  0.15983, accuracy:  0.94141], [G loss:  3.93781], Time[(Curr)0:00:27.832500, (Total)0:02:17.688133, (ETA)2022-10-07 21:48:44.699392]
  3000( 6/20), [D loss:  0.16827, accuracy:  0.93750], [G loss:  3.53357], Time[(Curr)0:00:27.90

: 

## Result

* Home PC
```log
Start Training - Data Shape: (60000, 28, 28, 1), Epochs: 10000, Batch Size: 128, Time: 2022-10-07 21:39:29.523741
  9500(19/20), [D loss:  0.07055, accuracy:  0.97266], [G loss:  5.02296], Time[(Curr)0:00:26.869979, (Total)0:08:37.478390, (ETA)2022-10-07 21:48:33.872137]
 10000(20/20), [D loss:  0.12159, accuracy:  0.96094], [G loss:  4.65250], Time[(Curr)0:00:26.344298, (Total)0:09:03.822688, (ETA)2022-10-07 21:48:33.346450]
End Training - Time: 2022-10-07 21:48:33.346513, Total Processing Time: 0:09:03.822772
```